In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from selenium.webdriver.common.keys import Keys
import time
import re
from tqdm.notebook import tqdm

In [2]:
import pandas as pd
import numpy as np
import random
from datetime import datetime ,timedelta , date

In [3]:
def get_date(tag):  # 뉴스의 날짜 post date 계산
    global current_time

    # posted_time을 None으로 초기화
    posted_time = None
    
    posted_time_str = tag.select('div.info > header > div > div > span.time')[0].text
    
    if 'ago' in posted_time_str:
        time_diff = re.search('\d+', posted_time_str).group()

        if 'day' in posted_time_str:
            posted_time = current_time - timedelta(days=int(time_diff))
        elif 'hour' in posted_time_str:
            posted_time = current_time - timedelta(hours=int(time_diff))
        elif 'min' in posted_time_str:
            posted_time = current_time - timedelta(minutes=int(time_diff))
        else:
            print('error from 1')

    else:
        post_date = re.search('\d+', posted_time_str).group()

        if 'tober' in posted_time_str:
            posted_time = datetime(2020, 10, int(post_date))
        elif 'tember' in posted_time_str:
            posted_time = datetime(2020, 9, int(post_date))
        else:
            print('error from 2')

    # posted_time이 None이 아닌 경우에만 isoformat 호출
    if posted_time:
        return posted_time.isoformat()
    else:
        return None  # 오류가 발생했을 경우 None 반환

In [30]:
browser_path = "/Users/boo/USA_President_Election/chromedriver" 
driver = webdriver.Chrome(browser_path)   
time.sleep(2)

In [32]:
current_time = datetime.now()
data=pd.DataFrame()
load_more = 5

In [33]:
driver.get('https://www.foxnews.com/category/columns/fox-news-poll')
time.sleep(2)

html = driver.page_source
soup = BeautifulSoup(html,'lxml')
articles = soup.select('#wrapper > div.page > div.page-content > div > main > section > div')[0].select('article')

news = [ {'headlines':article.select('div.info > header > h4 > a')[0].text,
         'date':get_date(article),
         'Press':'Fox',
         'Link':'https://www.foxnews.com'+article.select('div.info > header > h4 > a')[0].attrs['href'],
         'category':article.select('div.info > header > div > div > span.eyebrow > a')[0].text} for article in articles]
news_df = pd.DataFrame(news)
data = pd.concat([data,news_df])

error from 2
error from 2
error from 2
error from 2
error from 2
error from 2
error from 2
error from 2
error from 2
error from 2
error from 2
error from 2


In [36]:
data.tail()

,headlines,date,Press,Link,category
15,Fox News Poll: Harris bests Trump by 6 points ...,None,Fox,https://www.foxnews.com/official-polls/fox-new...,Fox News Poll
16,Fox News Poll: Dead heat between Harris and Tr...,None,Fox,https://www.foxnews.com/politics/fox-news-poll...,Fox News Poll
17,Fox News Poll: Voters cutting back on spendin...,None,Fox,https://www.foxnews.com/official-polls/fox-new...,Polls
18,Fox News Poll: Supreme Court approval rating d...,None,Fox,https://www.foxnews.com/official-polls/fox-new...,Fox News Poll
19,Fox News Poll: Approval of Congress lowest in ...,None,Fox,https://www.foxnews.com/official-polls/fox-new...,Fox News Elections Polls


In [13]:
def cbs_news(page_num):
    
    content=[]
    title=[]
    date=[]
    
    driver = webdriver.Chrome("/Users/boo/USA_President_Election/chromedriver")
    time.sleep(2)
    
    for i in range(1,page_num+1):
        
        driver.get(f'https://www.cbsnews.com/feature/election-2024/{i}/')
        time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, "lxml")
        soup=soup.select('div.col-8.nocontent')[0].select('section')
        
        try:
            for j in range(1,3): #26

                title.append(soup[0].select('h4.item__hed')[j-1].text.strip())
                date.append(soup[0].select('ul.item__metadata')[0].text)

                driver.find_element_by_xpath(f'//*[@id="component-election-2024"]/div/article[{j}]').click()
                time.sleep(2)

                html_content = driver.page_source
                soup_content = BeautifulSoup(html_content, "lxml")

                lis=soup_content.select('article')[0].select('p')

                if lis[0].text.strip()[:2]=='By':
                    con=''
                    for k in lis[2:]:
                        con+=k.text.strip() 

                else:
                    con=''
                    for k in lis:
                        con+=k.text.strip()


                content.append(con)
                driver.back()
                time.sleep(2)
        except:
            pass
    df=pd.DataFrame({'date':date,'title':title,'content':content})
    return df

In [14]:
data=cbs_news(1)

In [15]:
data.head()

,date,title,content
0,8H ago\n\n\n,"Larry Hogan says ""neither one of the two candi...",Washington — Former Maryland Gov. Larry Hogan ...
1,8H ago\n\n\n,Trump levies more personal attacks on Harris i...,"Updated on: September 28, 2024 / 10:06 PM EDT..."


In [16]:
#로봇이 아닙니다
def politico_news(page_num):
    
    content=[]
    title=[]
    date=[]

    driver = webdriver.Chrome("/Users/boo/USA_President_Election/chromedriver")
    time.sleep(2)

    for i in range(1,page_num+1):

        driver.get(f'https://www.politico.com/news/2024-elections/{i}')
        time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, "lxml")

        try:
            for j in range(1,2): #20

                driver.find_element_by_xpath(f'//*[@id="main"]/div/section[2]/div/div[1]').click()
                time.sleep(2)

                html_content = driver.page_source
                soup_content = BeautifulSoup(html_content, "lxml")

                lis=soup_content.select('article')[0].select('p')

                con=''
                for k in lis:
                    con+=k.text.strip() 

                content.append(con)
                driver.back()
                time.sleep(2)

                title.append(soup.select('ol.front-list')[0].select('h1')[j-1].text.strip())
                d=soup.select('ol.front-list')[0].select('p.timestamp')[j-1].text[:10]

                if d=='31/08/2024':
                    break

                date.append(d)

        except:
            pass

    df=pd.DataFrame({'date':date,'title':title,'content':content})
    return df

In [17]:
data=politico_news(2)

In [126]:
import time
#driver = webdriver.Chrome("/Users/boo/USA_President_Election/chromedriver")
driver.get("https://www.npr.org/sections/elections/") # get page

# n = 0
# while 1 :
#     driver.find_element_by_xpath(f'//*[@id="main-section"]/div[7]/button').click() # load 15 times
#     n += 1
#     if n > 1:
#         break
        
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
link_html = soup.select("div.item-info-wrap > div > h2 > a")  # get links
links = []
for link in link_html:
    links.append(link.get('href'))

    
df = []               # crawl data from every link
for link in links:
    driver.get(link)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    try : 
        title = soup.select("#main-section > article > div.storytitle > h1")[0].text # 기사제목
        date = soup.select("#story-meta > div.story-meta__one > div > div.dateblock > time > span")[0].text # 기사 날짜

        textall = soup.select("#storytext > p") # 기사 내용

        text = []
        for t in textall :
            text.append(t.text)
            
        topic = soup.select("#main-section > article > div.slug-wrap > h3 > a")[0].text # 주제

    except:
        pass

    article = {"Date" : date , "Press" : "NPR", "Topic": topic, "Title": title, "Text" : text} # 합치기 
    df.append(article)

In [127]:
df=pd.DataFrame(df)

In [128]:
df.tail()

,Date,Press,Topic,Title,Text
19,"September 24, 2024",NPR,"We, The Voters","MAGA hats, a rodeo, and carnival games: What w...","[Around 60 miles west of Las Vegas, Pahrump, N..."
20,"September 23, 2024",NPR,Politics,A GOP push to change how Nebraska awards its e...,[The Nebraska state lawmaker at the center of ...
21,"September 23, 2024",NPR,Politics,Trump says he wouldn't run for president agai...,[If former President Donald Trump loses the el...
22,"September 23, 2024",NPR,Politics,Here's what role Nevada — and its voters — cou...,[How Nevada swings on Election Day could tip t...
23,"September 22, 2024",NPR,Politics,"In one key swing state, both parties are court...","[In his 76 years, Phillip Barker has seen many..."


In [ ]:
npr = pd.DataFrame(df)
npr.dropna(inplace = True)
npr.drop_duplicates(["Title"])
npr[npr["Date"].str.contains('August')].index
npr = npr.loc[:340, :]
npr.to_csv("nprdata.csv")

fox, cbs, npr로 확인하기